### 0. Idea:
(1) Decompose the image into n (n>=1, e.g., 9) sub-images, so that we can dominate every analysis for each sub-image.

(2) For each sub-image, use sato-filter and thresholding by a specific percentile.

(3) (Optional) For each sub-image, use Gaussian smoothing to blur the image a little bit to remove some noises.

(4) For each sub-image, fix its broken vessel and (use clustering or other methods to) denoise

(5) Merge all the sub-images, and use a better way to eliminate the metallic.

In [1]:
import numpy as np
import pydicom
import os
import cv2
import glob
from PIL import Image, ImageDraw, ImageFilter
import matplotlib.pyplot as plt

from skimage.metrics import structural_similarity as ssim
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from skimage.feature import hessian_matrix, hessian_matrix_eigvals
from skimage.filters import sato, frangi, gaussian
from scipy.ndimage import gaussian_filter, label, binary_closing
from skimage import filters, measure, feature
from scipy.ndimage import binary_dilation, binary_fill_holes
from skimage.color import rgb2gray
from skimage.segmentation import active_contour

### 1. Decompose Image into Sub-images

In [2]:
def split_image(input_path, output_path, split_size): # split_size is the splitting number for both row and column, e.g., split_size = 3 -> 9 splitted images

    for img_path in glob.glob(os.path.join(input_path, '*.png')):
        img = Image.open(img_path)
        width, height = img.size  

        # calculate the width and height after splitting
        split_width = width // split_size
        split_height = height // split_size

        # store splitted images
        split_images = []

        # splitting images
        for i in range(split_size):  # row
            for j in range(split_size):  # column
                left = j * split_width
                upper = i * split_height
                right = left + split_width
                lower = upper + split_height

                # crop and split images
                split_img = img.crop((left, upper, right, lower))
                split_images.append(split_img)

        # save splitted images
        for idx, split_img in enumerate(split_images):
            img_num = os.path.splitext(os.path.basename(img_path))[0]
            filename = os.path.join(output_path, f'split_{img_num}_{idx + 1}.png') # save as split_0_1.png, split_0_2.png, ...
            cv2.imwrite(filename, np.array(split_img))

        print(f"Image has already been splitted into {split_size**2} sub-images")

In [3]:
input_path = "F:\\JJU Dataset\\CAG without CCTA\\Huang Rui Cheng\\20250103102526.486000\\2\\output"
output_path = "F:\\JJU Dataset\\CAG without CCTA\\Huang Rui Cheng\\20250103102526.486000\\2\\splitted_sub_images"
split_size = 2

split_image(input_path, output_path, split_size)

Image has already been splitted into 4 sub-images
Image has already been splitted into 4 sub-images
Image has already been splitted into 4 sub-images
Image has already been splitted into 4 sub-images
Image has already been splitted into 4 sub-images
Image has already been splitted into 4 sub-images
Image has already been splitted into 4 sub-images
Image has already been splitted into 4 sub-images
Image has already been splitted into 4 sub-images
Image has already been splitted into 4 sub-images
Image has already been splitted into 4 sub-images
Image has already been splitted into 4 sub-images
Image has already been splitted into 4 sub-images
Image has already been splitted into 4 sub-images
Image has already been splitted into 4 sub-images
Image has already been splitted into 4 sub-images
Image has already been splitted into 4 sub-images
Image has already been splitted into 4 sub-images
Image has already been splitted into 4 sub-images
Image has already been splitted into 4 sub-images


### 2. Use Sato-filter and Thresholding

In [13]:
def thresholding(sato_img, percentile):
    thres = np.percentile(sato_img, percentile)
    sato_img[sato_img <= thres] = 0
    sato_img[sato_img > thres] = 255

In [14]:
def gradient(img):
    grad_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)  # horizontal gradient
    grad_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)  # vertical gradient
    grad_norm = np.sqrt(grad_x**2 + grad_y**2)
    grad_direction = np.arctan2(grad_y, grad_x)
    return grad_norm, grad_direction

In [15]:
def gradient_direction(coords_li, direction_matrix):
    coord_direction_dict = {}

    for coord in coords_li:
        x, y = coord
        direction = direction_matrix[x, y]
        coord_direction_dict[coord] = direction
    return coord_direction_dict

In [16]:
def gradient_thres(img, grad_percentile, alpha=1, max_iter=3):
    edge_li = []
    grad_norm, grad_direction = gradient(img)
    grad_thres = np.percentile(img, grad_percentile)
    width, height = img.shape[0], img.shape[1]

    # find the coordinates that greater than the threshold
    coords = np.argwhere(grad_norm > grad_thres)
    # get the (x,y)-coordinate list
    for x, y in coords:
        img[int(x), int(y)] = 0 #let the edge = 0
        edge_li.append((int(x), int(y)))

    grad_norm, grad_direction = gradient(img) # recalculate the gradient direction after making edges = 0.
                       
    #edge_li = [(int(x), int(y)) for x, y in coords] #don't reset edge = 0, directly put edge into edge_list

    grad_direction_dict = gradient_direction(edge_li, grad_direction)

    enhance_li = list(edge_li) # copy edge_li

    for edge_point in edge_li:

        edge_point_x, edge_point_y = edge_point[0], edge_point[1]
        
        d = grad_direction_dict[edge_point]
        dx, dy = np.cos(d), np.sin(d)

        new_point_x, new_point_y = int(edge_point_x-alpha*dx), int(edge_point_y-alpha*dy)
        new_point = (new_point_x, new_point_y)
        iteration = 0
        
        while new_point not in edge_li and iteration<= max_iter:
            if not (0 <= new_point[0] < width and 0 <= new_point[1] < height):
                break
            else:
                enhance_li.append(new_point)
                new_point_x, new_point_y = int(new_point_x-alpha*dx), int(new_point_y-alpha*dy)
                new_point = (new_point_x, new_point_y)
                iteration = iteration+1

    unique_enhance_li = list(set(enhance_li))# eliminate repeated values in list

    for x, y in unique_enhance_li:
        img[x, y] = 0

    return img

In [17]:
def contour_filter(img, contour_percentile):
    contours = measure.find_contours(img, 0.8, fully_connected = 'low', positive_orientation = 'low')

    #fig, ax = plt.subplots()
    #ax.imshow(img, cmap=plt.cm.gray)
    #for contour in contours:
        #ax.plot(contour[:, 1], contour[:, 0], linewidth=2)

    #print(contour[:, 1], contour[:, 0])
    
    path_dict = {}
    
    for contour in contours:
        contour_li = contour.tolist()
        path_len = 0
        for i in range(len(contour)-1):
            path_len = path_len + np.linalg.norm(np.array(contour[i]) - np.array(contour[i+1]))# calculate path distance
        contour_tuple = tuple(map(tuple, contour)) # write contour into tuple form (list and array are non-hashable object)
        path_dict.update({contour_tuple:path_len})

    path_len_li = list(path_dict.values())
    path_thres = np.percentile(path_len_li, contour_percentile)# threshold

    contour_below_thres = []
    for contour_tuple, path_len in path_dict.items():
        if path_len <= path_thres:
            contour_below_thres.append(contour_tuple) 

    # create a full black bg, draw all the small noises with white color into the black bg
    width, height = img.shape
    image = Image.new('RGB', (width, height), color = 'black')
    draw = ImageDraw.Draw(image)
    
    for points in contour_below_thres:
        contour_below_thres_li = list(points)
        contour_below_thres_li += [contour_below_thres_li[0]]
        swapped_contours = [(y, x) for x, y in contour_below_thres_li] # swap (x,y) to (y,x), because the axis for image is invertible to matrix
        draw.polygon(swapped_contours, outline='white', fill='white')

    # use original image to minus the noise
    image = image.convert('L')
    new_img = img - np.array(image)   

    return new_img

    #filename = os.path.join(output_path, f'111.png')
    #plt.imsave(filename, new_img, cmap='gray')

#input_path = "F:\\JJU Dataset\\CAG without CCTA\\Huang Rui Cheng\\20250103102526.486000\\2\\sato_images (no subimage)\\sato_thres_split_24_1.png"
#output_path = "F:\\JJU Dataset\\CAG without CCTA\\Huang Rui Cheng\\20250103102526.486000\\2"
#img = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
#contour_filter(img,90, output_path)

In [18]:
def sato_filter(input_path, output_path, percentile, grad_percentile, contour_percentile):

    for img_path in glob.glob(os.path.join(input_path, '*.png')):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        #img = gradient_thres(img, grad_percentile, alpha=1, max_iter=3)
        sato_img = sato(img, sigmas=range(1, 15, 2), black_ridges=True, mode='reflect')
        smoothed_sato_img = gaussian(sato_img, sigma=1)
        thresholding(smoothed_sato_img, percentile) # don't write smoothed_sato_img = thresholding(...), it will lead a NoneType issue
        #contour_filter(smoothed_sato_img, contour_percentile)
        smoothed_sato_img = np.uint8(smoothed_sato_img)
        
        # 找到最大的连通区域（假设血管是最大的连通区域）
        contours, _ = cv2.findContours(smoothed_sato_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            # 找到面积最大的轮廓
            largest_contour = max(contours, key=cv2.contourArea)
            # 创建一个与原图大小相同的黑色图像
            vessel_mask = np.zeros_like(smoothed_sato_img)
            # 填充最大的轮廓，填充为1，然后对应元素相乘乘到原图
            cv2.drawContours(vessel_mask, [largest_contour], -1, 1, thickness=cv2.FILLED)
        else:
            vessel_mask = np.zeros_like(smoothed_sato_img)

        final_img = vessel_mask * smoothed_sato_img

        skeleton = cv2.ximgproc.thinning(final_img) #get the centerline of vessel
            
        img_name = int(os.path.splitext(os.path.basename(img_path))[0]) #remember to delete int and +1
        filename = os.path.join(output_path, f'sato_thres_{img_name}_4.png')
        plt.imsave(filename, final_img, cmap='gray')
        #plt.imsave(filename, smoothed_sato_img, cmap='gray')
        #cv2.imsave(filename, np.array(smoothed_sato_img)

In [8]:
#input_path = "F:\\JJU Dataset\\CAG without CCTA\\Huang Rui Cheng\\20250103102526.486000\\2\\splitted_sub_images"
input_path = f"F:\\JJU Dataset\\CAG without CCTA\\Huang Rui Cheng\\20250103102526.486000\\4\\demeaned_image"
#input_path = "F:\\ARCADE CAG Dataset\\arcade\\syntax\\test\\Dataset_vessel_test\\imagesTr_refine"
output_path = f"F:\\JJU Dataset\\CAG without CCTA\\Huang Rui Cheng\\20250103102526.486000\\three_views_sato"
#output_path = "F:\\ARCADE CAG Dataset\\arcade\\syntax\\test\\Dataset_vessel_test\\sato_label"
percentile = 90
grad_percentile = 1
contour_percentile = 90

sato_filter(input_path, output_path, percentile, grad_percentile, contour_percentile)

In [25]:
img_path = "F:\\JJU Dataset\\CAG without CCTA\\Huang Rui Cheng\\20250103102526.486000\\2\\demeaned_image\\30.png"
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
canny_img = feature.canny(img, sigma=1)
canny_img = canny_img.astype(np.uint8) * 1
sato_img = sato(img, sigmas=range(1, 15, 2), black_ridges=True, mode='reflect')
new_img = canny_img * sato_img
plt.imsave("intersection.png", new_img, cmap='gray')

### Evaluation

In [20]:
def crop_and_resize_image(input_path, output_path):#remove the black border of images

    count = 0
    
    for image in glob.glob(os.path.join(input_path, '*.png')):
    #image -> "F:\JJU Dataset\CAG without CCTA\Huang Rui Cheng\20250103102526.486000\1\output\xxx.png"
    
        with Image.open(image) as img:
            width, height = img.size

            border_width = int(width * 0.05)#remove 5% of width
            border_height = int(height * 0.05)#remove 5% of height

            cropped_img = img.crop((border_width, border_height, width - border_width, height - border_height))
        
       
            resized_img = cropped_img.resize((width, height), Image.Resampling.LANCZOS)#keep the images in the same size as before
        
        
            resized_img.save(output_path+f"\\{count}.png")

            print(output_path+f"\\{count}.png")

            count = count+1

In [21]:
input_path = "F:\\ARCADE CAG Dataset\\arcade\\syntax\\test\\Dataset_vessel_test\\imagesTr"
output_path = "F:\\ARCADE CAG Dataset\\arcade\\syntax\\test\\Dataset_vessel_test\\imagesTr_refine"
crop_and_resize_image(input_path, output_path)

F:\ARCADE CAG Dataset\arcade\syntax\test\Dataset_vessel_test\imagesTr_refine\0.png
F:\ARCADE CAG Dataset\arcade\syntax\test\Dataset_vessel_test\imagesTr_refine\1.png
F:\ARCADE CAG Dataset\arcade\syntax\test\Dataset_vessel_test\imagesTr_refine\2.png
F:\ARCADE CAG Dataset\arcade\syntax\test\Dataset_vessel_test\imagesTr_refine\3.png
F:\ARCADE CAG Dataset\arcade\syntax\test\Dataset_vessel_test\imagesTr_refine\4.png
F:\ARCADE CAG Dataset\arcade\syntax\test\Dataset_vessel_test\imagesTr_refine\5.png
F:\ARCADE CAG Dataset\arcade\syntax\test\Dataset_vessel_test\imagesTr_refine\6.png
F:\ARCADE CAG Dataset\arcade\syntax\test\Dataset_vessel_test\imagesTr_refine\7.png
F:\ARCADE CAG Dataset\arcade\syntax\test\Dataset_vessel_test\imagesTr_refine\8.png
F:\ARCADE CAG Dataset\arcade\syntax\test\Dataset_vessel_test\imagesTr_refine\9.png
F:\ARCADE CAG Dataset\arcade\syntax\test\Dataset_vessel_test\imagesTr_refine\10.png
F:\ARCADE CAG Dataset\arcade\syntax\test\Dataset_vessel_test\imagesTr_refine\11.png
F:

In [43]:
import os
import numpy as np
from PIL import Image

def calculate_iou(mask1, mask2):
    """
    计算两张二值化掩码图片的IoU。
    :param mask1: 第一张掩码图片的路径
    :param mask2: 第二张掩码图片的路径
    :return: IoU值
    """
    # 打开图片并转换为灰度模式
    img1 = Image.open(mask1).convert('L')
    img2 = Image.open(mask2).convert('L')
    
    # 将图片转换为numpy数组
    array1 = np.array(img1)
    array2 = np.array(img2)
    
    # 确保图片尺寸相同
    if array1.shape != array2.shape:
        raise ValueError("两张图片的尺寸不一致")
    
    # 将数组二值化（假设掩码图片已经是二值化的）
    array1 = (array1 > 128).astype(np.uint8)  # 假设阈值为128
    array2 = (array2 > 128).astype(np.uint8)
    
    # 计算交集和并集
    intersection = np.logical_and(array1, array2).sum()
    union = np.logical_or(array1, array2).sum()
    
    # 计算IoU
    iou = intersection / union if union != 0 else 0.0
    return iou

def calculate_folder_iou(folder1, folder2):
    """
    计算两个文件夹中所有匹配图片的IoU。
    :param folder1: 第一个文件夹路径
    :param folder2: 第二个文件夹路径
    :return: 所有图片的平均IoU
    """
    # 获取文件夹中的图片文件名列表
    files1 = sorted(os.listdir(folder1))
    files2 = sorted(os.listdir(folder2))
    
    if len(files1) != len(files2):
        raise ValueError("两个文件夹中的图片数量不一致")
    
    total_iou = 0.0
    iou_li = []
    for file1, file2 in zip(files1, files2):
        # 构造图片路径
        path1 = os.path.join(folder1, file1)
        path2 = os.path.join(folder2, file2)
        
        # 计算单张图片的IoU
        iou = calculate_iou(path1, path2)
        total_iou += iou
        #print(f"{file1} 和 {file2} 的IoU: {iou:.4f}")
        iou_li.append(iou)
    
    # 计算平均IoU
    avg_iou = total_iou / len(files1)
    #max_iou = max(iou_li)
    print(f"Average IoU: {avg_iou:.4f}")
    return avg_iou

# 使用示例
folder1 = "F:\\ARCADE CAG Dataset\\arcade\\syntax\\test\\Dataset_vessel_test\\labelsTr" # 替换为第一个文件夹路径
folder2 = "F:\\ARCADE CAG Dataset\\arcade\\syntax\\test\\Dataset_vessel_test\\sato_label"  # 替换为第二个文件夹路径

calculate_folder_iou(folder1, folder2)

Average IoU: 0.3445


np.float64(0.34453195095204864)

In [53]:
import os
import numpy as np
from PIL import Image

def calculate_f_score(mask1, mask2):
    """
    计算两张二值化掩码图片的 F-Score。
    :param mask1: 第一张掩码图片的路径
    :param mask2: 第二张掩码图片的路径
    :return: F-Score
    """
    # 打开图片并转换为灰度模式
    img1 = Image.open(mask1).convert('L')
    img2 = Image.open(mask2).convert('L')
    
    # 将图片转换为二值化 NumPy 数组（假设阈值为 128）
    array1 = np.array(img1) > 128
    array2 = np.array(img2) > 128
    
    # 确保图片尺寸相同
    if array1.shape != array2.shape:
        raise ValueError("两张图片的尺寸不一致")
    
    # 计算 TP, FP, FN
    TP = np.sum(array1 & array2)  # 真正例
    FP = np.sum(array1 & ~array2)  # 假正例
    FN = np.sum(~array1 & array2)  # 假负例
    
    # 计算 Precision 和 Recall
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    
    # 计算 F-Score
    f_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    return f_score

def calculate_folder_f_score(folder1, folder2):
    """
    计算两个文件夹中所有匹配图片的 F-Score。
    :param folder1: 第一个文件夹路径
    :param folder2: 第二个文件夹路径
    :return: 所有图片的平均 F-Score
    """
    # 获取文件夹中的图片文件名列表
    files1 = sorted(os.listdir(folder1))
    files2 = sorted(os.listdir(folder2))
    
    if len(files1) != len(files2):
        raise ValueError("两个文件夹中的图片数量不一致")

    f_li = []
    total_f_score = 0.0
    for file1, file2 in zip(files1, files2):
        # 构造图片路径
        path1 = os.path.join(folder1, file1)
        path2 = os.path.join(folder2, file2)
        
        # 计算单张图片的 F-Score
        f_score = calculate_f_score(path1, path2)
        total_f_score += f_score
        f_li.append(f_score)
        #print(f"{file1} 和 {file2} 的 F-Score: {f_score:.4f}")
    
    # 计算平均 F-Score
    avg_f_score = total_f_score / len(files1)
    #max_f_score = max(f_li)
    print(f"所有图片的平均 F-Score: {avg_f_score:.4f}")
    return avg_f_score

# 使用示例
folder1 = "F:\\ARCADE CAG Dataset\\arcade\\syntax\\test\\Dataset_vessel_test\\labelsTr" # 替换为第一个文件夹路径
folder2 = "F:\\ARCADE CAG Dataset\\arcade\\syntax\\test\\Dataset_vessel_test\\sato_label"  # 替换为第二个文件夹路径

calculate_folder_f_score(folder1, folder2)

所有图片的平均 F-Score: 0.5125


np.float64(0.5124935122710759)

### Exclude Watermark

In [5]:
import pydicom
import cv2
import numpy as np
import os

# 读取DICOM文件
def read_dicom_frames(dicom_file):
    dataset = pydicom.dcmread(dicom_file)
    # 获取像素数据
    pixel_array = dataset.pixel_array
    return pixel_array

# 将DICOM帧保存为MP4视频
def save_dicom_to_mp4(dicom_frames, output_mp4, fps=30):
    # 获取帧的尺寸
    height, width = dicom_frames.shape[1], dicom_frames.shape[2]
    # 定义视频编码格式
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    # 创建视频写入对象
    out = cv2.VideoWriter(output_mp4, fourcc, fps, (width, height), isColor=False)
    # 将每一帧写入视频
    for frame in dicom_frames:
        out.write(frame)
    out.release()

# 示例使用
dicom_file = 'F:\\JJU Dataset\\CAG without CCTA\\Huang Rui Cheng\\20250103102526.486000\\2\\00572087_20250103_1_18.dcm'  # 替换为你的DICOM文件路径
output_mp4 = 'F:\\JJU Dataset\\CAG without CCTA\\Huang Rui Cheng\\20250103102526.486000\\2\\output_video.mp4'

# 读取DICOM帧
dicom_frames = read_dicom_frames(dicom_file)

# 保存为MP4视频
save_dicom_to_mp4(dicom_frames, output_mp4)

In [27]:
import cv2
import numpy as np
import os

# 初始化背景减除器
backSub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)

# 打开视频文件
cap = cv2.VideoCapture('F:\\JJU Dataset\\CAG without CCTA\\Huang Rui Cheng\\20250103102526.486000\\2\\output_video.mp4')  # 替换为你的MP4文件路径

# 获取第一帧作为初始背景
ret, frame = cap.read()
if ret:
    # 转换为灰度图像
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # 将第一帧添加到背景模型中
    backSub.apply(gray_frame, learningRate=0)

# 创建输出文件夹
output_folder = "F:\\JJU Dataset\\CAG without CCTA\\Huang Rui Cheng\\20250103102526.486000\\2\\dcm"
os.makedirs(output_folder, exist_ok=True)

# 处理每一帧
frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 转换为灰度图像
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 应用背景减除
    fg_mask = backSub.apply(gray_frame, learningRate=0.001)

    # 对前景掩码进行形态学操作以去除噪声
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    #fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
    #fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)

    # 对前景掩码进行平滑处理
    #fg_mask = cv2.GaussianBlur(fg_mask, (5, 5), 0)

    # 应用阈值处理以获取二值化图像
    #_, fg_mask = cv2.threshold(fg_mask, 127, 255, cv2.THRESH_BINARY)

    # 找到最大的连通区域（假设血管是最大的连通区域）
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        # 找到面积最大的轮廓
        largest_contour = max(contours, key=cv2.contourArea)
        # 创建一个与原图大小相同的黑色图像
        vessel_mask = np.zeros_like(fg_mask)
        # 填充最大的轮廓
        cv2.drawContours(vessel_mask, [largest_contour], -1, 255, thickness=cv2.FILLED)
    else:
        vessel_mask = np.zeros_like(fg_mask)

    # 保存血管掩码到指定路径
    output_path = os.path.join(output_folder, f'vessel_mask_{frame_count:04d}.png')
    cv2.imwrite(output_path, fg_mask)

    # 按下 'q' 键退出循环
    if cv2.waitKey(30) & 0xff == ord('q'):
        break

    frame_count += 1

# 释放资源并关闭窗口
cap.release()
cv2.destroyAllWindows()

### 3. Merge Subimages

In [21]:
def merge_images(input_path, output_path, split_size):
    
    # 创建一个空白的512x512图片
    result_img = Image.new('RGB', (512, 512))

    num_img = int(len(glob.glob(os.path.join(input_path, '*.png')))/(split_size**2))

    for i in range(num_img):
        split_images = []
        for j in range(1,split_size**2+1):
            img = Image.open(input_path+f"\\sato_thres_split_{i}_{j}.png")
            split_images.append(img)
        
        # 计算每一份的大小
        split_width = 512 // split_size
        split_height = 512 // split_size

        # 拼接图片
        for k in range(split_size):  # 行
            for j in range(split_size):  # 列
                left = j * split_width
                upper = k * split_height
                result_img.paste(split_images[k * split_size + j], (left, upper))
                
        filename = os.path.join(output_path, f"merged_img_{i}.png") 
        cv2.imwrite(filename, np.array(result_img))
    print("Merged image has already been saved.")

In [22]:
input_path = "F:\\JJU Dataset\\CAG without CCTA\\Huang Rui Cheng\\20250103102526.486000\\2\\sato_thres_subimages"
output_path = "F:\\JJU Dataset\\CAG without CCTA\\Huang Rui Cheng\\20250103102526.486000\\2\\sato_thres_img"
split_size = 2

merge_images(input_path, output_path, split_size)

Merged image has already been saved.


### test code

In [82]:
def contour_filter(img, contour_percentile, output_path):
    contours = measure.find_contours(img, 0.8, fully_connected = 'low', positive_orientation = 'low')

    #fig, ax = plt.subplots()
    #ax.imshow(img, cmap=plt.cm.gray)
    #for contour in contours:
        #ax.plot(contour[:, 1], contour[:, 0], linewidth=2)

    #print(contour[:, 1], contour[:, 0])
    
    path_dict = {}
    
    for contour in contours:
        contour_li = contour.tolist()
        path_len = 0
        for i in range(len(contour)-1):
            path_len = path_len + np.linalg.norm(np.array(contour[i]) - np.array(contour[i+1]))
        contour_tuple = tuple(map(tuple, contour))
        path_dict.update({contour_tuple:path_len})

    path_len_li = list(path_dict.values())
    path_thres = np.percentile(path_len_li, contour_percentile)

    contour_below_thres = []
    for contour_tuple, path_len in path_dict.items():
        if path_len <= path_thres:
            contour_below_thres.append(contour_tuple)
    print(len(contour_below_thres))

    image = Image.new('RGB', (512, 512), color = 'black')
    draw = ImageDraw.Draw(image)
    
    for points in contour_below_thres:
        contour_below_thres_li = list(points)
        contour_below_thres_li += [contour_below_thres_li[0]]
        swapped_contours = [(y, x) for x, y in contour_below_thres_li] # swap (x,y) to (y,x), because the axis for image is invertible to matrix
        draw.polygon(swapped_contours, outline='white', fill='white')

    image = image.convert('L')
    new_img = img - np.array(image)   

    filename = os.path.join(output_path, f'000.png')
    plt.imsave(filename, new_img, cmap='gray')

input_path = "F:\\JJU Dataset\\CAG without CCTA\\Huang Rui Cheng\\20250103102526.486000\\2\\sato_images (no subimage)\\sato_thres_split_24_1.png"
output_path = "F:\\JJU Dataset\\CAG without CCTA\\Huang Rui Cheng\\20250103102526.486000\\2"
img = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
contour_filter(img,90, output_path)

47
